## stacking 集成算法-只有付费记录：次级学习器融合-最佳模型
- 4 ExtraTreesRegressor
- 5 Gradient Boosting Regressor
- 6 RandomForestRegressor
- 7 BaggingRegressor

In [1]:
# 导入必要的工具包
import numpy as np #用于数值计算
import pandas as pd #用于数据表处理，数据文件读写
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from sklearn import ensemble

import matplotlib.pyplot as plt #用于绘图：分析结果的可视化。
%matplotlib inline

In [2]:
def get_train_val(val_id):
    '''获取训练集和校验集 
    val_id = 0,1,2 #当前校验集编号 '''
    print('val_id=%s'% (val_id))
    df_val = train_list[val_id]
    train_tmp = []

    for i in range(1, train_part_num):
        cur_id = (val_id + i) % 5
        print('cur_id=%s'% (cur_id))
        train_tmp.append(train_list[cur_id])
    
#     只生成一次，节省内存 
    df_train = pd.concat(train_tmp)
    print('df_train.shape=%s  df_val.shape=%s'% (df_train.shape, df_val.shape))
    return df_train, df_val

In [3]:
Y_Standard = True
path_train = '../data/new/train=predict/'
path_test = '../data/new/test=predict/'

val_path = '../data/val/second/' #保存校验集测试结果
val_name = '%sval=stacking-%s-%s.csv'
summit_path = '../data/summit/stacking-second/' #保存提交文件，测试集测试结果 
out_name = '%ssummit=stacking-%s-%s.csv' # 生成的结果文件名称
base = 'second' #学习器级别 

train_part_num = 5 #训练集拆分个数
target = 'prediction_pay_price'
user_id = 'user_id'

## 指定读取数据类型，节省内存

In [4]:
col_int32 = ['user_id']
col_float32 = ['pred_0', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5', 'pred_6', 'pred_7']

In [5]:
# 指定读取数据类型，节省内存
dtype_test = {}
for col in col_int32:
    dtype_test[col] = np.int32
for col in col_float32:
    dtype_test[col] = np.float32
# dtype_test

In [6]:
dtype_train = dtype_test.copy()
dtype_train[target] = np.float32
# dtype_train

## 读取数据，一次读取多次使用，不同组合 

In [7]:
train_list = []
for i in range(train_part_num):
    cur_id = i + 1
    train_list.append(pd.read_csv('%strain=pred-1-col=9-target-%s.csv' % (path_train, cur_id), dtype=dtype_train))
train_list[0].head()

,user_id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,prediction_pay_price
0,332994,0.000000,0.000079,0.0,0.349532,0.000068,0.0,0.000000,0.000071,0.0
1,2186887,0.164846,0.000099,0.0,0.349532,0.004556,0.0,0.108522,0.000158,0.0
2,383991,0.163507,0.000097,0.0,0.349532,0.000122,0.0,0.052701,0.000087,0.0
3,102315,0.157621,0.000141,0.0,0.349532,0.000161,0.0,0.057641,0.000138,0.0
4,225310,0.000000,0.000147,0.0,0.349532,0.000736,0.0,0.013085,0.000055,0.0


## 5折交叉验证

In [8]:
# 根据校验集编号 0，1,2,3,4 ，获取训练集和校验集
df_train, df_val = get_train_val(4)

# 从原始数据中分离输入特征x和输出y
train_y = df_train[target].values
train_X = df_train.drop([target, user_id], axis = 1)
val_y = df_val[target].values
val_X = df_val.drop([target, user_id], axis = 1)

val_id=4
cur_id=0
cur_id=1
cur_id=2
cur_id=3
df_train.shape=(1830405, 10)  df_val.shape=(457602, 10)


In [9]:
# 数据标准化：分别初始化对特征和目标值的标准化器 
ss_X = StandardScaler()
ss_y = StandardScaler()

# 分别对训练和测试数据的特征以及目标值进行标准化处理
train_X_ss = ss_X.fit_transform(train_X)
val_X_ss = ss_X.transform(val_X)

if Y_Standard:
    train_y_ss = ss_y.fit_transform(train_y.reshape(-1, 1))
    val_y_ss = ss_y.transform(val_y.reshape(-1, 1))

# 需要转换为一维数组 
train_y_ss_1d = train_y_ss.reshape(train_y_ss.shape[0])
val_y_ss_1d = val_y_ss.reshape(val_y_ss.shape[0])

In [10]:
def inverse_StandardScaler(predict):
    ''' 标准化数据还原  '''
    if Y_Standard:
        predict = ss_y.inverse_transform(predict)
        print('predict = ', predict)
    return predict

def generate_summit(predict):
    ''' 输出预测后的数据 '''
    testPredict = test.copy()
    testPredict[target] = predict
    testPredict = testPredict[[user_id, target]]
#     应该过滤掉负数
    testPredict[target] = testPredict[target].apply(lambda x: x if x > 0 else 0)
    return testPredict

def generate_val_predict(predict):
    ''' 生成校验集预测后的数据 '''
    testPredict = df_val.copy()
    testPredict[target] = predict
    testPredict = testPredict[[user_id, target]]
#     应该过滤掉负数
    testPredict[target] = testPredict[target].apply(lambda x: x if x > 0 else 0)
    return testPredict

## 模型训练
###  4. ExtraTreesRegressor

In [11]:
%%time
model = ensemble.ExtraTreesRegressor(n_estimators=5, min_samples_split=40, min_samples_leaf=8, n_jobs=-1)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 21.4 s

Wall time: 8.07 s


In [12]:
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss)
scoreVal = model.score(val_X_ss, val_y_ss)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss, predict_train) ** 0.5
predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss, predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# n_estimators=16, min_samples_split=80, min_samples_leaf=8 
# n_estimators=12, min_samples_split=80, min_samples_leaf=8
# n_estimators=12, min_samples_split=40, min_samples_leaf=8
# n_estimators=10, min_samples_split=40, min_samples_leaf=8
# n_estimators=5, min_samples_split=40, min_samples_leaf=8
# The score: Train=0.575943581110089  Val=0.4301712408468244
# The RMSE: Train=0.6511960342288877  Val=0.9656998080873251

The score: Train=0.5689679197417558  Val=0.43236165459505854
The RMSE: Train=0.6565304651570552  Val=0.6629262797520857


### 保存校验集测试结果

In [13]:
print('predict_val= ', predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'ExtraTreesRegressor'), index=False)
valPredict.head()

predict_val=  [-0.02016223 -0.0201605  -0.02016223 ... -0.02016223 -0.02016095
 -0.02016109]
predict =  [-8.71155108e-08  1.56228844e-04 -8.71155108e-08 ... -8.71155108e-08
  1.16253095e-04  1.03597073e-04]


,user_id,prediction_pay_price
0,1987367,0.000000
1,2292622,0.000156
2,466954,0.000000
3,1008130,0.000104
4,9759,0.000054


## 加载测试数据

In [14]:
test = pd.read_csv(path_test + 'test=pred-1-col=9.csv', dtype=dtype_test)
test_X = test.drop([user_id], axis = 1)
test_X_ss = ss_X.transform(test_X)
test.head()

,user_id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7
0,14933,0.084183,0.000180,0.007868,0.406852,1.084812e-04,0.010231,1.650189,0.000162
1,14934,0.059209,0.000117,0.000000,0.406852,2.858160e-05,0.000000,0.671875,0.000123
2,14935,0.392662,0.000134,0.000000,0.406852,4.165320e-05,0.000000,0.931189,0.000121
3,14936,1.513114,0.006305,0.235470,0.406852,1.391971e-08,0.235131,1.364640,0.000070
4,14937,0.000000,0.000141,0.000000,0.406852,3.695815e-05,0.000000,0.599589,0.000145


In [15]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)
# lasso_y_predict_test =  [0.4556051 0.4556051 0.4556051 ... 0.4556051 0.4556051 0.4556051]

model_y_predict_test =  [-0.02016223 -0.02016109 -0.02016127 ... -0.02015968 -0.02016115
 -0.02016223]
predict =  [-8.71155108e-08  1.02814643e-04  8.69058242e-05 ...  2.30583422e-04
  9.75775104e-05 -8.71155108e-08]


In [16]:
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'ExtraTreesRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,0.000000
1,14934,0.000103
2,14935,0.000087
3,14936,0.008250
4,14937,0.000207


## 5 GradientBoostingRegressor

In [17]:
%%time
model = ensemble.GradientBoostingRegressor(learning_rate=0.25, n_estimators=53, min_samples_split=40, min_samples_leaf=9, alpha=0.999)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 1min 42s

Wall time: 1min 47s


In [18]:
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss_1d)
scoreVal = model.score(val_X_ss, val_y_ss_1d)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

model_predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss_1d, model_predict_train) ** 0.5
model_predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss_1d, model_predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# learning_rate=0.25, n_estimators=20, min_samples_split=40, min_samples_leaf=9, alpha=0.99
# learning_rate=0.25, n_estimators=25, min_samples_split=40, min_samples_leaf=9, alpha=0.999
# learning_rate=0.25, n_estimators=50, min_samples_split=40, min_samples_leaf=9, alpha=0.999
# learning_rate=0.25, n_estimators=53, min_samples_split=40, min_samples_leaf=9, alpha=0.999
# The score: Train=0.7344110187155188  Val=0.45512882728699006
# The RMSE: Train=0.5153531712937626  Val=0.9443149791592277

The score: Train=0.7168464594185933  Val=0.40118358341354904
The RMSE: Train=0.5321218405509504  Val=0.6808888493521704


### 保存校验集测试结果

In [19]:
print('model_predict_val= ', model_predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(model_predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'GradientBoostingRegressor'), index=False)
valPredict.head()

model_predict_val=  [-0.01775406 -0.01775406 -0.01775406 ... -0.01775406 -0.01775406
 -0.01775406]
predict =  [0.21801319 0.21801319 0.21801319 ... 0.21801319 0.21801319 0.21801319]


,user_id,prediction_pay_price
0,1987367,0.218013
1,2292622,0.218013
2,466954,0.218013
3,1008130,0.218013
4,9759,0.218013


In [20]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)

model_y_predict_test =  [-0.01775406 -0.01775406 -0.01775406 ... -0.01775406 -0.01775406
 -0.01775406]
predict =  [0.21801319 0.21801319 0.21801319 ... 0.21801319 0.21801319 0.21801319]


In [21]:
# ridge_y_predict_test = move_up(ridge_y_predict_test)
#输出预测后的数据
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'GradientBoostingRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,0.218013
1,14934,0.218013
2,14935,0.218013
3,14936,0.218013
4,14937,0.218013


## 6 RandomForestRegressor

In [22]:
%%time
model = ensemble.RandomForestRegressor(n_estimators=20, min_samples_split=40, min_samples_leaf=8, n_jobs=-1)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 3min 24s

Wall time: 1min 40s


In [23]:
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss_1d)
scoreVal = model.score(val_X_ss, val_y_ss_1d)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

model_predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss_1d, model_predict_train) ** 0.5
model_predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss_1d, model_predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# n_estimators=16, min_samples_split=80, min_samples_leaf=8
# n_estimators=16, min_samples_split=40, min_samples_leaf=8
# n_estimators=50, min_samples_split=40, min_samples_leaf=8
# n_estimators=20, min_samples_split=40, min_samples_leaf=8
# The score: Train=0.6141246079194357  Val=0.377331913649869
# The RMSE: Train=0.621188587420921  Val=1.009481272914462

The score: Train=0.5952262839501917  Val=0.4313191206886586
The RMSE: Train=0.6362184192775102  Val=0.6635347711346112


### 保存校验集测试结果

In [24]:
print('model_predict_val= ', model_predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(model_predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'RandomForestRegressor'), index=False)
valPredict.head()

model_predict_val=  [-0.02016067 -0.0201615  -0.02016039 ... -0.01992452 -0.02016128
 -0.0201614 ]
predict =  [1.40786746e-04 6.62469575e-05 1.66540792e-04 ... 2.15203213e-02
 8.60558643e-05 7.50686851e-05]


,user_id,prediction_pay_price
0,1987367,0.000141
1,2292622,0.000066
2,466954,0.000167
3,1008130,0.000075
4,9759,0.000078


In [25]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)

model_y_predict_test =  [-0.02016082 -0.02016058 -0.02016102 ... -0.02015869 -0.02016024
 -0.02015988]
predict =  [0.00012769 0.00014957 0.00010984 ... 0.00032073 0.00018014 0.00021255]


In [26]:
# ridge_y_predict_test = move_up(ridge_y_predict_test)
#输出预测后的数据
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'RandomForestRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,0.000128
1,14934,0.000150
2,14935,0.000110
3,14936,0.000205
4,14937,0.000195


## 7 BaggingRegressor

In [27]:
%%time
model = ensemble.BaggingRegressor(n_jobs=3, n_estimators=10)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 1min 51s

Wall time: 1min 9s


In [28]:
%%time
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss_1d)
scoreVal = model.score(val_X_ss, val_y_ss_1d)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

model_predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss_1d, model_predict_train) ** 0.5
model_predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss_1d, model_predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# n_estimators=20 
# n_estimators=15
# n_estimators=10
# The score: Train=0.9111143426339843  Val=0.3714285903170899
# The RMSE: Train=0.29813692693889515  Val=1.0142552740940023

The score: Train=0.9113453689534009  Val=0.3813879064997065
The RMSE: Train=0.29774933468350706  Val=0.6920517575399638
Wall time: 7.62 s


### 保存校验集测试结果

In [29]:
print('model_predict_val= ', model_predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(model_predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'BaggingRegressor'), index=False)
valPredict.head()

model_predict_val=  [-0.02016108 -0.02016101 -0.02016084 ... -0.02015917 -0.02016141
 -0.02016147]
predict =  [1.04253787e-04 1.10034506e-04 1.25778367e-04 ... 2.77036143e-04
 7.42412013e-05 6.87385967e-05]


,user_id,prediction_pay_price
0,1987367,0.000104
1,2292622,0.000110
2,466954,0.000126
3,1008130,0.000069
4,9759,0.000098


In [30]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)

model_y_predict_test =  [-0.02016031 -0.02016017 -0.02016088 ... -0.02015808 -0.02016051
 -0.02016092]
predict =  [0.00017359 0.00018628 0.00012235 ... 0.00037572 0.00015528 0.00011825]


In [31]:
# ridge_y_predict_test = move_up(ridge_y_predict_test)
#输出预测后的数据
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'BaggingRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,0.000174
1,14934,0.000186
2,14935,0.000122
3,14936,0.000159
4,14937,0.000195


## 7-1 BaggingRegressor的代码最佳示例
https://www.helplib.com/Python-Module-Examples/article_66154